## Microsoft-Movie-Studio-Box-Office-Analysis
---

**Author**:[Oscar Mulei](mailto:omulei@gmail.com)

## Overview
***
This project provides a comprehensive analysis of box office performance for movies. The goal is to offer valuable insights to Microsoft's newly established movie studio. The analysis covers various aspects of movies, such as genres, ratings, budgets, and release dates, to uncover the factors that contribute to a successful box office performance.
***

## Business Problem
***
Microsoft, recognizing the success of other big companies in creating original video content, is eager to enter the movie industry. However, despite their eagerness, they lack experience in movie creation. The challenge lies in understanding the current movie landscape and identifying the types of films that perform well at the box office. This project aims to explore these aspects, and its findings will guide the decision-making process for Microsoft's new movie studio, helping to decide what type of films to create.

***

## Data
***
The project leverages several datasets, providing comprehensive movie-related information. Each movie has a unique ID associated with its [intake](link-to-intake-dataset) and [outcome](link-to-outcome-dataset) data. The datasets offer valuable data points, such as movie titles, genres, average ratings, domestic and foreign gross revenues, and release years.

Given the business problem, we recognize the need to incorporate additional data to provide more comprehensive insights. This includes information on movie genre, budget and box office revenue, director and cast, runtime, and regional and language specifics. However, our current dataset includes the movie name, release year, and IMDB rating. Future enhancements of this analysis may include these additional data points.
***

In [1]:
# Import standard packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

%matplotlib inline

In [2]:
# Define a dictionary of file paths and corresponding DataFrame names
files = {
    'bom_movie_gross': 'bom.movie_gross.csv.gz',
    'rt_movie_info': 'rt.movie_info.tsv.gz',
    'rt_reviews': 'rt.reviews.tsv.gz',
    'tmdb_movies': 'tmdb.movies.csv.gz',
    'tn_movie_budgets': 'tn.movie_budgets.csv.gz'
}

# Load all the files into DataFrames
df_dict = {name: pd.read_csv(path, sep='\t' if path.endswith('.tsv.gz') else ',', encoding='ISO-8859-1') for name, path in files.items()}

# Connect to the SQLite database
conn = sqlite3.connect('movies.db')

# Write the dataframes to SQLite tables
for table_name, df in df_dict.items():
    df.to_sql(table_name, conn, if_exists='replace', index=False)

# Verify the tables in the SQLite database
tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
tables

C:\Users\HP\anaconda3\envs\learn-env\lib\site-packages\pandas\core\generic.py:2605: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


[('bom_movie_gross',),
 ('rt_movie_info',),
 ('rt_reviews',),
 ('tmdb_movies',),
 ('tn_movie_budgets',)]

## Data Preparation
***
The first step in preparing the data for analysis is to examine the structure of the tables we've loaded into the SQLite database and identify common keys that can be used to merge them.

I will start by loading each table into a pandas DataFrame and examining the first few rows and the column names. From this, we can get a sense of the structure of the data and how the tables might be related.
***

In [3]:
# Connect to the SQLite database
conn = sqlite3.connect('movies.db')

# Get the list of tables
tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
table_names = [table[0] for table in tables]

# Load each table into a pandas DataFrame and display the first few rows
for table_name in table_names:
    df = pd.read_sql(f'SELECT * FROM {table_name}', conn)
    print(f'--- {table_name} ---')
    print(df.head())
    print('\n')

--- bom_movie_gross ---
                                         title studio  domestic_gross  \
0                                  Toy Story 3     BV     415000000.0   
1                   Alice in Wonderland (2010)     BV     334200000.0   
2  Harry Potter and the Deathly Hallows Part 1     WB     296000000.0   
3                                    Inception     WB     292600000.0   
4                          Shrek Forever After   P/DW     238700000.0   

  foreign_gross  year  
0     652000000  2010  
1     691300000  2010  
2     664300000  2010  
3     535700000  2010  
4     513900000  2010  


--- rt_movie_info ---
   id                                           synopsis rating  \
0   1  This gritty, fast-paced, and innovative police...      R   
1   3  New York City, not-too-distant-future: Eric Pa...      R   
2   5  Illeana Douglas delivers a superb performance ...      R   
3   6  Michael Douglas runs afoul of a treacherous su...      R   
4   7                             

---
Now that we have a better understanding of the structure of the tables, let's proceed with these cleaning steps.

---

In [4]:
# Clean the `rt_reviews` DataFrame
rt_reviews_df = df_dict['rt_reviews']

# Convert `date` to datetime format
rt_reviews_df['date'] = pd.to_datetime(rt_reviews_df['date'], errors='coerce')

# Assign the cleaned DataFrame back to the dictionary
df_dict['rt_reviews'] = rt_reviews_df

# Show the first few rows of the cleaned `rt_reviews` DataFrame
rt_reviews_df.head()

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,2018-11-10
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,2018-05-23
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,2018-01-04
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,2017-11-16
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,2017-10-12


In [5]:
# Let's handle missing values for each DataFrame
for name, df in df_dict.items():
    # Print the initial number of missing values
    print(f"Initial missing values in {name}:")
    print(df.isnull().sum())
    
    # For simplicity, let's drop rows with any missing values
    # In a real-world scenario, you might want to handle missing values in a more sophisticated way
    # such as imputation, interpolation, etc.
    df = df.dropna()
    
    # Print the final number of missing values
    print(f"Final missing values in {name}:")
    print(df.isnull().sum())
    print("\n-------------------------------\n")
    
    # Assign the cleaned DataFrame back to the dictionary
    df_dict[name] = df

Initial missing values in bom_movie_gross:
title                0
studio               5
domestic_gross      28
foreign_gross     1350
year                 0
dtype: int64
Final missing values in bom_movie_gross:
title             0
studio            0
domestic_gross    0
foreign_gross     0
year              0
dtype: int64

-------------------------------

Initial missing values in rt_movie_info:
id                 0
synopsis          62
rating             3
genre              8
director         199
writer           449
theater_date     359
dvd_date         359
currency        1220
box_office      1220
runtime           30
studio          1066
dtype: int64
Final missing values in rt_movie_info:
id              0
synopsis        0
rating          0
genre           0
director        0
writer          0
theater_date    0
dvd_date        0
currency        0
box_office      0
runtime         0
studio          0
dtype: int64

-------------------------------

Initial missing values in rt_revi

Missing values in all DataFrames have been handled by dropping rows with any missing values. Here is the summary:

1. `bom_movie_gross`: Dropped rows with missing values in `studio`, `domestic_gross`, and `foreign_gross`.
2. `rt_movie_info`: Dropped rows with missing values in `synopsis`, `rating`, `genre`, `director`, `writer`, `theater_date`, `dvd_date`, `currency`, `box_office`, `runtime`, and `studio`.
3. `rt_reviews`: Dropped rows with missing values in `review`, `rating`, `critic`, and `publisher`.
4. `tmdb_movies`: No missing values were present.
5. `tn_movie_budgets`: No missing values were present.
